In [1]:
#加载tokenizer
from transformers import AutoTokenizer
pretrained_model_name_or_path = r'C:\Users\peixi\Downloads\Huggingface\model\hflchinese-roberta-wwm-ext'
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path)

import torch
from datasets import load_from_disk

class Dataset(torch.utils.data.Dataset):
    def __init__(self, split, max_samples=None):
        self.dataset = load_from_disk(r'c:\users\peixi\downloads\huggingface\data\Chnsentialz')[split]
        if max_samples:
            self.dataset = self.dataset.select(list(range(min(max_samples, len(self.dataset)))))

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        review = self.dataset[i]['review']
        label = self.dataset[i]['label']
        return review, label

dataset = Dataset('train', 140000) 
len(dataset), dataset[20]

#定义计算设备
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

# 数据整理函数
def collate_fn(data):
    sents = [i[0] for i in data]
    labels = [i[1] for i in data]
    # 编码
    data = tokenizer.batch_encode_plus(batch_text_or_text_pairs=sents, truncation=True, padding='max_length', max_length=500, return_tensors='pt', return_length=True)
    # input_ids：编码之后的数字
    # attention_mask：补零的位置是0, 其他位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']
    labels = torch.LongTensor(labels)
    # 把数据移动到计算设备上
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    token_type_ids = token_type_ids.to(device)
    labels = labels.to(device)
    return input_ids, attention_mask, token_type_ids, labels



In [2]:
#数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=32,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

len(loader)

4375

In [3]:
#加载模型
from transformers import BertModel
pretrained_model_name_or_path = r'C:\Users\peixi\Downloads\Huggingface\model\hflchinese-roberta-wwm-ext\trained'
pretrained = BertModel.from_pretrained(pretrained_model_name_or_path)


In [4]:
#不训练预训练模型,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)

In [5]:
#设定计算设备
pretrained.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [6]:
# 预训练模型试算
for input_ids, attention_mask, token_type_ids, labels in loader:
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    token_type_ids = token_type_ids.to(device)
    labels = labels.to(device)

    out = pretrained(input_ids=input_ids,
                     attention_mask=attention_mask,
                     token_type_ids=token_type_ids)

    print(out.last_hidden_state.shape)
    break  # 退出循环

torch.Size([32, 500, 768])


In [7]:
#定义下游任务模型
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(in_features=768, out_features=2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        #使用预训练模型抽取数据特征
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                             attention_mask=attention_mask,
                             token_type_ids=token_type_ids)

        #对抽取的特征只取第一个字的结果做分类即可
        out = self.fc(out.last_hidden_state[:, 0])

        out = out.softmax(dim=1)

        return out


model = Model()

#设定计算设备
model.to(device)

#试算
model(input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids=token_type_ids).shape

torch.Size([32, 2])

In [8]:
#训练
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR

def train():
    # 定义优化器
    optimizer = optim.AdamW(model.parameters(), lr=5e-4)

    # 定义loss函数
    criterion = torch.nn.CrossEntropyLoss()

    # 定义学习率调节器
    scheduler = LambdaLR(optimizer, lr_lambda=lambda step: 0.98 ** step)  # 使用LambdaLR作为学习率调节器

    # 模型切换到训练模式
    model.train()

    # 按批次遍历训练集中的数据
    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):

        # 模型计算
        out = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        # 计算loss并使用梯度下降法优化模型参数
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        # 输出各项数据的情况，便于观察
        if i % 60 == 0:
            out = out.argmax(dim=1)
            accuracy = (out == labels).sum().item() / len(labels)
            lr = optimizer.param_groups[0]['lr']
            print(i, loss.item(), lr, accuracy)

train()

0 0.6955153942108154 0.00049 0.59375
60 0.47216925024986267 0.00014580103991913902 0.84375
120 0.5114313960075378 4.3383557635719116e-05 0.8125
180 0.45744362473487854 1.2908913915672958e-05 0.875
240 0.5374895930290222 3.841087904350535e-06 0.78125
300 0.49398624897003174 1.1429277772961926e-06 0.8125
360 0.503663957118988 3.4008175200460194e-07 0.84375
420 0.501954197883606 1.0119239408121162e-07 0.8125
480 0.6337202191352844 3.0110114875404044e-08 0.65625
540 0.459847629070282 8.959359307997236e-09 0.84375
600 0.5095941424369812 2.665885518602479e-09 0.71875
660 0.4408225119113922 7.932426141175809e-10 0.875
720 0.48663631081581116 2.3603183274799914e-10 0.75
780 0.4138438403606415 7.023201360954808e-11 0.90625
840 0.4768548309803009 2.0897756367116802e-11 0.8125
900 0.5736762285232544 6.218193082249874e-12 0.71875
960 0.3871147930622101 1.8502428934898528e-12 0.9375
1020 0.4521421194076538 5.50545587701668e-13 0.84375
1080 0.41976290941238403 1.638165698159117e-13 0.9375
1140 0.545

In [16]:
#测试
def test():
    #定义测试数据集加载器
    loader_test = torch.utils.data.DataLoader(dataset=Dataset('test'),
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    #下游任务模型切换到运行模式
    model.eval()
    correct = 0
    total = 0
    sentence_count = 0  # 用于记录句子数量

    #增加输出前5句的结果并与真实的label进行比较
    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader_test):

        #计算
        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)

        #统计正确率
        out = out.argmax(dim=1)
        correct += (out == labels).sum().item()
        total += len(labels)

        #输出前5句的结果并与真实的label进行比较
        for j in range(len(input_ids)):  # 遍历当前batch的每个句子
            # Decode input_ids
            decoded_input_ids = tokenizer.decode(input_ids[j], skip_special_tokens=True)
            print("Review:", decoded_input_ids)  # 输出 input_ids 的 decode 结果
            print("Prediction:", out[j].item())  # 输出预测结果
            print("True label:", labels[j].item())  # 输出真实标签
            sentence_count += 1  # 句子数量加1
            if sentence_count == 5:  # 当输出了5条句子后，退出循环
                break

        if sentence_count == 5:  # 当输出了5条句子后，退出外层循环
            break

    print("Accuracy for the first 5 sentences:", correct / total)

test()

Review: 第 二 次 穿 裤 子 就 起 毛 了 看 来 是 买 到 假 劣 产 品 了
Prediction: 0
True label: 0
Review: 我 买 浅 卡 色 和 宝 蓝 发 过 来 的 是 卡 其 色
Prediction: 0
True label: 0
Review: 最 近 落 魄 了 很 久 真 的 是 失 魂 落 魄 银 行 用 普 通 白 信 封 寄 给 我 的 新 信 用 卡 都 被 我 忽 略 到 了 角 落 隔 了 二 十 多 天 才 好 容 易 从 一 厚 沓 邮 件 中 翻 出 来 我 真 是 该 打 闭 门 思 过 中
Prediction: 0
True label: 0
Review: 质 量 不 错 面 料 挺 好 的 很 划 算
Prediction: 1
True label: 1
Review: 双 鱼 又 开 始 多 愁 善 感 了
Prediction: 0
True label: 0
Accuracy for the first 5 sentences: 0.8125
